In [26]:
!pip install -q cassio datasets langchain openai tiktoken PyPDF2 langchain_community langchain_openai

In [27]:
from langchain.vectorstores.cassandra import Cassandra
from langchain.indexes.vectorstore import VectorStoreIndexWrapper
#from langchain.llms import OpenAI
from langchain_openai import OpenAI
from langchain.embeddings import OpenAIEmbeddings
from datasets import load_dataset

In [16]:
from dotenv import load_dotenv
load_dotenv()
import os
from PyPDF2 import PdfReader

In [13]:
os.environ["ASTRA_DB_ID"]="53ed12c4-7e9d-4a83-8b47-9e44a9a27ed2"
os.environ["OPENAI_API_KEY"]=os.getenv("OPENAI_API_KEY")
os.environ["ASTRA_DB_TOKEN"]=os.getenv("ASTRA_DB_TOKEN")

In [18]:
pdfreader=PdfReader("budget-2024.pdf")

In [20]:
from typing_extensions import Concatenate
raw_text=""
for i, page in enumerate(pdfreader.pages):
    content=page.extract_text()
    if content:
        raw_text+=content

In [24]:
import cassio
cassio.init(token=os.environ["ASTRA_DB_TOKEN"], database_id=os.environ["ASTRA_DB_ID"])

In [28]:
llm=OpenAI()
embeddings=OpenAIEmbeddings()


/tmp/ipykernel_163/1187853183.py:2: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embeddings=OpenAIEmbeddings()


In [30]:
astra_vector_store=Cassandra(
    embedding=embeddings,
    table_name="pdf_embeddings_budget",
    session=None,
    keyspace=None)

In [33]:
from langchain.text_splitter import CharacterTextSplitter
text_splitter=CharacterTextSplitter(
    separator="\n",
    chunk_size=1000,
    chunk_overlap=200
)
texts=text_splitter.split_text(raw_text)

In [36]:
texts[:1]

['©His Majesty the King in Right of Canada (2024 ) \nAll rights reserved\nAll requests for permission to reproduce this document \nor any part thereof shall be addressed to the Department of Finance Canada.\nThis document is available at www.Canada.ca/Budget\nCette publication est aussi disponible en français.\nF1-24/3E-PDF\nCat No.: 1719-7740\nIn case of discrepancy between the printed version and the electronic version, \nthe electronic version will prevail.   i Deputy Prime Minister’s Foreword  \nA fair chance to build a good middle class life —to do as well as your parents, or \nbetter —that’s the promise of Canada. For too many, especially for younger \nCanadians, that promise is at risk.  \nWe have a plan to fix that. We have a plan to build a Canada that works better \nfor you, where you can get ahead, where your hard work pays off, where you \ncan buy a home —where you have  a fair chance at a good middle class life.  \nFirst, we’re building more affordable homes.  Because the 

In [42]:
#load into vector store
astra_vector_store.add_texts(texts)
astra_vector_index=VectorStoreIndexWrapper(vectorstore=astra_vector_store)

In [43]:
query_text="what is the hydrogen investment tax credit?"
answer=astra_vector_index.query(query_text, llm=llm).strip()
print(answer)

The hydrogen investment tax credit is a tax credit that will be available as of March 28, 2023 for eligible investments related to clean hydrogen production. It is one of the investment tax credits that the government is prioritizing in introducing legislation for.
